In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import os
import climate_learn as cl
from climate_learn.transforms import Mask, Denormalize
from climate_learn.data import ERA5ToPrism
import torch

In [3]:
dm = ERA5ToPrism(
    os.path.join(os.environ["PRISM_DIR"], "era5_cropped"),
    os.path.join(os.environ["PRISM_DIR"], "prism_processed")
)

In [4]:
dm.setup()

In [5]:
mask = Mask(dm.get_out_mask().to(device="cuda:0"))
denorm = Denormalize(dm)

In [6]:
denorm_mask = lambda x: denorm(mask(x))

In [27]:
for batch in dm.train_dataloader():
    x, y, in_vars, out_vars= batch
    break

In [45]:
trainer = cl.Trainer(accelerator="gpu", devices=[0])

Global seed set to 0


In [46]:
nearest = cl.load_downscaling_module(
    data_module=dm,
    preset="nearest-interpolation",
    train_target_transform=mask,
    val_target_transform=[denorm_mask, denorm_mask, denorm_mask, mask],
    test_target_transform=[denorm_mask, denorm_mask, denorm_mask]
)

trainer.test(nearest, dm)

Loading preset: nearest-interpolation
Using preset optimizer
Using preset learning rate scheduler
Loading training loss: mse
Using custom training transform
Loading validation loss: rmse
Loading validation loss: pearson
Loading validation loss: mean_bias
Loading validation loss: mse
Using custom validation transform
Using custom validation transform
Using custom validation transform
Using custom validation transform
Loading test loss: rmse
Loading test loss: pearson
Loading test loss: mean_bias
Using custom test transform
Using custom test transform
Using custom test transform


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test/mean_bias:aggregate  │   -0.047938872989860604   │
│ test/mean_bias:daily_tmax │   -0.047938872989860604   │
│  test/pearson:aggregate   │    0.8995036909952759     │
│  test/pearson:daily_tmax  │    0.8995036909952759     │
│    test/rmse:aggregate    │    3.1585565831850593     │
│   test/rmse:daily_tmax    │    3.1585565831850593     │
└───────────────────────────┴───────────────────────────┘

In [48]:
bilinear = cl.load_downscaling_module(
    data_module=dm,
    preset="bilinear-interpolation",
    train_target_transform=mask,
    val_target_transform=[denorm_mask, denorm_mask, denorm_mask, mask],
    test_target_transform=[denorm_mask, denorm_mask, denorm_mask]
)

trainer.test(bilinear, dm)

Loading preset: bilinear-interpolation
Using preset optimizer
Using preset learning rate scheduler
Loading training loss: mse
Using custom training transform
Loading validation loss: rmse
Loading validation loss: pearson
Loading validation loss: mean_bias
Loading validation loss: mse
Using custom validation transform
Using custom validation transform
Using custom validation transform
Using custom validation transform
Loading test loss: rmse
Loading test loss: pearson
Loading test loss: mean_bias
Using custom test transform
Using custom test transform
Using custom test transform


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test/mean_bias:aggregate  │    0.32576804131092735    │
│ test/mean_bias:daily_tmax │    0.32576804131092735    │
│  test/pearson:aggregate   │    0.9195437123293715     │
│  test/pearson:daily_tmax  │    0.9195437123293715     │
│    test/rmse:aggregate    │    2.8106137483363076     │
│   test/rmse:daily_tmax    │    2.8106137483363076     │
└───────────────────────────┴───────────────────────────┘